# DSCI 525: Web and Cloud Computing

## Milestone 1: Tackling Big Data on Computer

### Group 13
Authors: Ivy Zhang, Mike Lynch, Selma Duric, William Xu

## Table of contents

- [Download the data](#1)
- [Combining data CSVs](#2)
- [Load the combined CSV to memory and perform a simple EDA](#3)
- [Perform a simple EDA in R](#4)
- [Reflection](#5)

### Imports

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np
import pyarrow.feather as feather
from memory_profiler import memory_usage

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

## Download the data <a name="1"></a>

1. Download the data from figshare to local computer using the figshare API.
2. Extract the zip file programmatically.

In [3]:
# Attribution: DSCI 525 lecture notebook
# Necessary metadata
article_id = 14096681  # unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]    

In [5]:
%%time
files_to_dl = ["data.zip"]  
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 3.65 s, sys: 3.7 s, total: 7.35 s
Wall time: 1min 17s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 18.2 s, sys: 3.69 s, total: 21.9 s
Wall time: 28.8 s


## Combining data CSVs <a name="2"></a>

1. Use one of the following options to combine data CSVs into a single CSV (Pandas, Dask). **We used the option of Pandas**.
2. When combining the csv files, we added extra column called "model" that identifies the model (we get this column populated from the file name eg: for file name "SAM0-UNICON_daily_rainfall_NSW.csv", the model name is SAM0-UNICON)
3. Compare run times and memory usages of these options on different machines within the team, and summarize observations.

In [7]:
%%time
%memit
# Shows time that regular python takes to merge file
# Join all data together
## here we are using a normal python way of merging the data 
# use_cols = ["time", "lat_min", "lat_max", "lon_min","lon_max","rain (mm/day)"]
files = glob.glob('figsharerainfall/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'^[^_]+(?=_)', file)[0])
                for file in files)
              )
df.to_csv("figsharerainfall/combined_data.csv")

peak memory: 95.41 MiB, increment: 0.26 MiB
CPU times: user 7min 28s, sys: 31 s, total: 7min 59s
Wall time: 9min 17s


In [8]:
feather.write_feather(df, "figsharerainfall/combined_data.feather")

In [9]:
%%sh
du -sh figsharerainfall/combined_data.csv

6.6G	figsharerainfall/combined_data.csv


In [10]:
%%time
df = pd.read_csv("figsharerainfall/combined_data.csv")

CPU times: user 1min 12s, sys: 25.2 s, total: 1min 37s
Wall time: 1min 51s


In [11]:
print(df.shape)

(62513863, 7)


In [12]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,figsharerainfall/MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,figsharerainfall/MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,figsharerainfall/MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,figsharerainfall/MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,figsharerainfall/MPI-ESM-1-2-HAM


**Summary of run times and memory usages:**

***William***
- Combining files: 
    - peak memory: 95.41 MiB, increment: 0.26 MiB
    - CPU times: user 7min 28s, sys: 31 s, total: 7min 59s
    - Wall time: 9min 17s
- Reading the combined file:
    - Wall time: 1min 51s


Feel free to add your run times and memory usages and list here (we meant to compare these metrics)

## Load the combined CSV to memory and perform a simple EDA <a name="3"></a>

## Perform a simple EDA in R <a name="4"></a>

## Reflection <a name="5"></a>